Problem 1

In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [9]:
dist = pd.read_csv('mwm_dist.csv')
dist.drop(['Status'], axis=1, inplace=True)
dist = dist[['OriginInd', 'DestinationInd', 'Distance', 'Time']]
dist.sort_values(by=['OriginInd', 'DestinationInd'], inplace=True)
dist.reset_index(drop=True, inplace=True)
hosp = pd.read_csv('mwm_hosp.csv')

In [10]:
dist, hosp

(       OriginInd  DestinationInd  Distance  Time
 0              0               0         0     0
 1              0               1     34949  1775
 2              0               2     32397  1802
 3              0               3     63731  2829
 4              0               4     49704  2707
 ...          ...             ...       ...   ...
 13919        117             113     42257  2569
 13920        117             114      8183  1221
 13921        117             115     50469  2718
 13922        117             116     14849  1785
 13923        117             117         0     0
 
 [13924 rows x 4 columns],
      HospitalID                                               Name   latitude  \
 0             0                                   MWM Headquarters  34.213938   
 1             1                             Verdugo Hills Hospital  34.204530   
 2             2  Barlow Respiratory Hospital - Los Angeles (Mai...  34.075693   
 3             3                        Gle

In [11]:
# Part 1a
a = dist.loc[dist['OriginInd'] == 0, 'Distance'].idxmax()
print(f'The hospital farthest from the depot is {hosp.loc[a, "Name"]} with a distance of {dist.loc[a, "Distance"]:.2f} km')

The hospital farthest from the depot is Antelope Valley Hospital with a distance of 90167.00 km


In [12]:
# Part 1b
b = dist.loc[(dist['OriginInd'] == 0) & (dist['DestinationInd'] != 0), 'Distance'].idxmin()
print(f'The hospital farthest from the depot is {hosp.loc[b, "Name"]} with a distance of {dist.loc[b, "Distance"]:.2f} km')

The hospital farthest from the depot is Mission Community Hospital with a distance of 2141.00 km


In [13]:
# Part 1c
def transport_cost(distance):
    return 0.1 * distance + 5000

def heuristic1(dist, hosp):
    hospitals = list(hosp.HospitalID.values)
    visited = [0]
    distance = 0
    hospitals.pop(0)
    for i in range(len(hospitals)):
        next_idx = dist.loc[(dist['DestinationInd'] != 0) & (dist['OriginInd'] == visited[-1]) & (dist['DestinationInd'].isin(hospitals)) & (dist['OriginInd'] != dist['DestinationInd']), 'Distance'].idxmin()
        next_hosp = dist.loc[next_idx, 'DestinationInd']
        distance += dist.loc[next_idx, 'Distance']
        visited.append(next_hosp)
        hospitals.pop(hospitals.index(next_hosp))
    distance += dist.loc[(dist['OriginInd'] == visited[-1]) & (dist['DestinationInd'] == 0), 'Distance'].values[0]
    return distance, visited

a, b = heuristic1(dist, hosp)

print(f'The total transport cost is {transport_cost(a):.2f}')

The total transport cost is 94553.90


In [14]:
# Part 1d
def get_distances(dist, sequence):
    distance = 0
    for i in range(len(sequence) - 1):
        distance += dist.loc[(dist['OriginInd'] == sequence[i]) & (dist['DestinationInd'] == sequence[i+1]), 'Distance'].values[0]
    return distance

np.random.seed(50)
cost = []

for i in range(100):
    sequence = np.random.permutation(range(1, 118))
    sequence = np.insert(sequence, 0, 0)
    sequence = np.append(sequence, 0)
    distance = get_distances(dist, sequence)
    cost.append(transport_cost(distance))

print(f'The average cost of the heuristic is {np.mean(cost):.2f}')
print(f'The best/lowest cost of the heuristic is {np.min(cost):.2f}')

The average cost of the heuristic is 488934.47
The best/lowest cost of the heuristic is 438397.50


Part 2a

\begin{align}
& \underset{\mathbf{x}}{\text{minimize}} & & \sum_{i=1}^n \sum_{ \substack{j=1\\ j\neq i} }^n  d_{i,j} x_{i,j} \;\;\;\; \text{[Objective Function]} \\
& \text{subject to} & & \sum_{\substack{j = 1\\j \neq i}}^n x_{i,j} = 1, \quad \forall\ i\ \in \{1,\dots,n\} \;\;\;\; \text{[Each hospital has exactly one outgoing path]} \\
& & & \sum_{\substack{j = 1\\j \neq i}}^n x_{j,i} = 1, \quad \forall\ i\ \in \{1,\dots,n\} \;\;\;\; \text{[Each hospital has exactly one incoming path]} \\
& & & \sum_{i \in S} \sum_{ \substack{j \in S\\j\neq i} } x_{i,j} \leq |S| - 1, \quad \forall \ S \subsetneq \{1,\dots,n\}, \;\;\;\; \text{[Subtour Elimination Constraint]} \\
& & & x_{i,j} \in \{0,1\}, \quad \forall \ i,j \in \{1,\dots,n\}, \ i \neq j. \;\;\;\; \text{[Binary Constraint]} 
\end{align}


where decision variables are given as follows:

$d_{ij}$ is the travel distance from hospital $i$ to hospital $j$.

$x_{ij} = 1$ if if the path from hospital $i$ to hospital $j$ is included in the tour

$x_{ij} = 0$ otherwise

to minimize total distance travelled.

In [15]:
# Part 2b
origin_dest_zipped = list(zip(dist['OriginInd'], dist['DestinationInd']))
travel_dist_dict = dict( zip(origin_dest_zipped, dist['Distance']) )
od_pairs = travel_dist_dict.keys()
nHospitals = len(hosp)

def getSubtours(sequence):
    subtour_list = []
    unvisited = list(range(nHospitals))
    while ( len(unvisited) > 0 ):
        node = unvisited.pop()
        subtour = []
        subtour.append(node)
        next_node = list(filter(lambda t: t[0] == node, sequence))[0][1]
        while (next_node in unvisited):
            subtour.append(next_node)
            unvisited.remove(next_node)
            next_node = list(filter(lambda t: t[0] == next_node, sequence))[0][1]
        subtour_list.append(subtour)
    return subtour_list

def eliminateSubtours(model, where):
    if (where == GRB.Callback.MIPSOL):
        x_val = model.cbGetSolution(x)
        sequence = [ (i,j) for (i,j) in od_pairs if x_val[i,j] > 0.5]
        subtour_list = getSubtours(sequence)
        if (len(subtour_list) > 1):
            for subtour in subtour_list:
                model.cbLazy( sum(x[i,j] for i in subtour for j in subtour if j != i) <= (len(subtour) - 1))

from gurobipy import *

m = Model()
m.Params.LogToConsole = 0

x = m.addVars(od_pairs, vtype = GRB.BINARY)

for i in range(nHospitals):
    m.addConstr( sum(x[i,j] for j in range(nHospitals) if j != i ) == 1)
    m.addConstr( sum(x[j,i] for j in range(nHospitals) if j != i ) == 1)

m.setObjective(sum( travel_dist_dict[i,j] * x[i,j] for (i,j) in od_pairs ), GRB.MINIMIZE)

m.update()
m.params.LazyConstraints = 1
m.optimize(eliminateSubtours)

sequence = [ (i,j) for (i,j) in od_pairs if x[i,j].x > 0.5]
subtour_list = getSubtours(sequence)
complete_tour = subtour_list[0]
complete_tour.append( complete_tour[0] )
print("Complete tour: ", complete_tour)
print("Total cost: ", transport_cost(m.ObjVal))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-13


TypeError: unsupported operand type(s) for -: 'bool' and 'NoneType'

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src/gurobipy/callback.pxi", line 208, in gurobipy.CallbackClass.callback
  File "/var/folders/v0/zsfpxhv11w53vl0c98l6h02m0000gq/T/ipykernel_77557/1865809705.py", line 29, in eliminateSubtours
  File "src/gurobipy/model.pxi", line 6992, in gurobipy.Model.cbLazy
TypeError: unsupported operand type(s) for -: 'bool' and 'NoneType'


Complete tour:  [117, 32, 5, 58, 1, 84, 103, 90, 31, 33, 41, 98, 16, 71, 0, 20, 95, 73, 43, 25, 63, 74, 21, 15, 52, 83, 19, 68, 28, 6, 112, 65, 70, 116, 24, 78, 62, 17, 96, 55, 114, 12, 9, 51, 45, 57, 108, 35, 4, 48, 14, 69, 89, 111, 80, 11, 26, 34, 27, 42, 23, 61, 46, 86, 37, 53, 110, 30, 18, 113, 13, 40, 8, 85, 109, 36, 66, 101, 88, 22, 87, 115, 3, 82, 7, 81, 50, 104, 59, 100, 49, 102, 97, 99, 39, 64, 107, 67, 47, 76, 94, 92, 38, 44, 79, 60, 54, 77, 93, 29, 75, 2, 105, 106, 91, 56, 10, 72, 117]
Total cost:  77578.89999999978


In [16]:
# Part 2c
for hospital in complete_tour[complete_tour.index(0) + 1 : complete_tour.index(0) + 6]:
    print(hosp.loc[hospital, "Name"])

Mission Community Hospital
Mission City Community Network Inc
Northridge Hospital Medical Center
West Hills Hospital and Medical Center
Kaiser Permanente - Woodland Hills Medical Center
